# Импорт необходимых для работы кода библиотек

In [15]:
import pandas as pd
import xlsxwriter
import json
import re
import xml.etree.ElementTree as ET
import io

# Настройки (язык регистрочувствительный)

In [5]:
# укажи полный путь до xml файла без расширения .bpmn
# либо просто название файла, если он находится в той же папке, что и этот скрипт
file_name = r'vitank3'

# укажи префикс bpmn:definitions xmlns:bpmn в фигурных скобках
xmls_bpmn_prefix = "{http://www.omg.org/spec/BPMN/20100524/MODEL}"

# укажи префикс bpmn:definitions xmlns:camunda в фигурных скобках
xmlns_camunda_prefix = "{http://camunda.org/schema/1.0/bpmn}"

# укажи список тасок для рассмотрения
task_list = ['serviceTask', 'scriptTask', 'userTask', 'callActivity', 
             'startEvent', 'boundaryEvent', 'intermediateCatchEvent', 'endEvent',
             'linkEventDefinition', 'executionListener'
#              'exclusiveGateway', 'parallelGateway', 
            ]

# укажи как сохранять input/output. Как словарь, или дублировать строки
io_like_dict = False

# Код

In [26]:
def get_df_from_xml(xml_string) -> pd.DataFrame:
    
    # should write autodetection of bpmn:, camunda: prefixes with some regex
    pass

    #create root
    root = ET.fromstring(xml_string)
    
    records = dict()
    index = 0
    
    for child in list(root.iter()):
        # we need to consider only process subtags (i suppose)
        if child.tag == xmls_bpmn_prefix + 'process':
            for child in list(child.iter()):
#                 print(child)
                if child.tag in [xmls_bpmn_prefix + task_ent for task_ent in task_list]:
                    
                    # collect first level data
                    records_ = dict()
                    records_['name'] = child.attrib.get('name', '-')
                    records_['id'] = child.attrib.get('id', '-')
                    records_['type'] = (child.tag.replace(xmls_bpmn_prefix,'') 
                                        if xmls_bpmn_prefix in child.tag 
                                        else child.tag)
                    records_['delegate'] = child.attrib.get("{}class".format(xmlns_camunda_prefix), '') 
                    
                    # collect input/output parameters description and docs
                    input_ = dict()
                    ouptut_ = dict()
                    for child in list(child.iter()):
                        if child.tag == xmls_bpmn_prefix + 'documentation':
                            records_['documentation'] = child.text
                        if child.tag == xmlns_camunda_prefix + 'inputOutput':
                            for child in list(child.iter()):
                                if child.tag == xmlns_camunda_prefix + 'inputParameter':
                                    if child.attrib.get('name', '-'):
                                        input_[child.attrib['name']] = child.text
                                if child.tag == xmlns_camunda_prefix + 'outputParameter':
                                    if child.attrib.get('name', '-'):
                                        ouptut_[child.attrib['name']] = child.text
                    
                    if io_like_dict == True:
                        records_['inputParameter'] = input_
                        records_['outputParameter'] = ouptut_
                    else:
                        records_['inputParameter'] = '\n'.join([f"{key} = {input_[key]}" for key in input_])
                        records_['outputParameter'] = '\n'.join([f"{key} = {ouptut_[key]}" for key in ouptut_])
                    
                    records[index] = records_
                    index += 1
       
    df = pd.DataFrame.from_dict(records, orient='index')

    return df


with open(file=file_name + ".bpmn", mode="r", encoding='utf-8') as file:
    
    # get data as string
    xml_string = file.read().replace('/n', '').rstrip()

    df = get_df_from_xml(xml_string)

    with pd.ExcelWriter(file_name + ".xlsx", engine="xlsxwriter") as writer:
        writer.book.formats[0].set_text_wrap() 
        writer.book.formats[0].set_align('left')
        writer.book.formats[0].set_align('top')
        df.to_excel(writer, index=False)
    
    print('Файл сохранен!')

Файл сохранен!
